## Death in Fiction

### Imports and Setup

In [ ]:
import os
import pandas as pd
import re
import warnings

from pyvis.network import Network

warnings.filterwarnings('ignore')

In [ ]:
kills = pd.DataFrame(columns=['universe', 'culprit', 'direct count', 'indirect count', 'count', 'victims'])
deaths = pd.DataFrame(columns=['universe', 'victim', 'culprits', 'description', 'notes'])
edges = pd.DataFrame(columns=['universe', 'culprit', 'victim', 'direction', 'description', 'notes'])

### Collection

In [ ]:
for path, subdirs, files in os.walk('Universes'):
    for name in files:
        file = path + '\\' + name
        universe = file.split('\\')[1::2][0]

        tsv = pd.read_csv(file, sep='\t')

        for i in range(len(tsv)):
            victim = tsv['victim'][i]
            direct_culprits = str(tsv['direct culprit(s)'][i]).split(', ')
            indirect_culprits = str(tsv['indirect culprit(s)'][i]).split(', ')
            description = tsv['description'][i]
            notes = tsv['notes'][i]

            if direct_culprits == ['nan']:
                direct_culprits = []
            if indirect_culprits == ['nan']:
                indirect_culprits = []

            culprits = direct_culprits + indirect_culprits
            if culprits == ['nan']:
                culprits = []

            for i in range(len(culprits)):
                if culprits[i] in direct_culprits:
                    row = [universe, culprits[i], 1, 0, 1, victim]
                    kills.loc[len(kills)] = row
                elif culprits[i] in indirect_culprits:
                    row = [universe, culprits[i], 0, 1, 1, victim]
                    kills.loc[len(kills)] = row

                if culprits[i] in direct_culprits:
                    row = [universe, culprits[i], victim, 'direct', description, notes]
                    edges.loc[len(edges)] = row
                elif culprits[i] in indirect_culprits:
                    row = [universe, culprits[i], victim, 'indirect', description, notes]
                    edges.loc[len(edges)] = row

            if culprits == []:
                row = [universe, '', victim, '', description, notes]
                edges.loc[len(edges)] = row

            row = [universe, victim, culprits, description, notes]
            deaths.loc[len(deaths)] = row

### Organisation

In [ ]:
kills = kills.groupby(by=['universe', 'culprit']).aggregate({'direct count':sum, 'indirect count':sum, 'count':sum, 'victims':', '.join})
kills.reset_index(inplace=True)
print('----- the 10 Deadliest Killers  -----')
kills.drop('victims', axis=1).sort_values(by='count', ascending=False).head(10)

In [ ]:
print('----- Deadliest Killer per Universe -----')
kills.drop('victims', axis=1).sort_values(by='count', ascending=False).drop_duplicates('universe')

In [ ]:
deaths_count = pd.DataFrame(deaths.groupby(by=['universe', 'victim']).size()).reset_index()
deaths_count.columns = ['universe', 'character', 'deaths']

kills_count = pd.DataFrame(kills.groupby(by=['universe', 'culprit']).sum('count')).reset_index()
kills_count.drop(['direct count', 'indirect count'], axis=1, inplace=True)
kills_count.columns = ['universe', 'character', 'kills']

summary = pd.merge(deaths_count, kills_count, left_on=['universe', 'character'], right_on=['universe', 'character'], how='outer')
summary.fillna(0, inplace=True)
summary.sort_values(by=['universe', 'character'], inplace=True)

summary['ratio'] = summary['kills'] / summary['deaths']
summary.loc[summary['deaths'] == 0, 'ratio'] = 0

summary['score'] = 0

for i in range(len(kills)):
    universe = kills.iloc[i]['universe']
    culprit = kills.iloc[i]['culprit']
    count = kills.iloc[i]['count']
    victims = kills.iloc[i]['victims'].split(', ')

    summary.loc[(summary['universe'] == universe) & (summary['character'] == culprit), 'score'] += count
    victims = list(set(victims))

    for victim in victims:
        if victim != '' and victim != culprit:
            try:
                score = summary.loc[(summary['universe'] == universe) & (summary['character'] == victim), 'kills'].values[0]
            except IndexError:
                score = 0
            summary.loc[(summary['universe'] == universe) & (summary['character'] == culprit), 'score'] += score

summary['transitivity'] = summary['score'] - summary['kills']
print("----- the 10 Deadliest 'Killer Killers' -----")
summary.sort_values(by='transitivity', ascending=False).head(10)

In [ ]:
print('----- Deadliest Universes -----')
pd.DataFrame(deaths.groupby(by=['universe']).size().sort_values(ascending=False)).reset_index().rename(columns={0:'deaths'})

In [ ]:
deaths.to_csv('deaths.tsv', sep='\t', index=False)
kills.to_csv('kills.tsv', sep='\t', index=False)
summary.to_csv('summary.tsv', sep='\t', index=False)

### Visualisation

In [ ]:
edges['culprit'] += " [" + edges['universe'] + "]"
edges.loc[edges['culprit'].str.startswith(' ['), 'culprit'] = ''
edges['victim'] += " [" + edges['universe'] + "]"
edges['color'] = pd.Series()

for i in range(len(edges)):
    notes = str(edges.iloc[i]['notes']).split(', ')

    if 'VAR' in notes:  # Variable
        dependent = True
    else:
        dependent = False

    if 'DNC' in notes:  # Does Not Count
        definitive = False
    else:
        definitive = True

    if not definitive:
        edges.iloc[i, 6] = '0_green'
    elif dependent and definitive:
        edges.iloc[i, 6] = '1_purple'
    if not dependent and definitive:
        edges.iloc[i, 6] = '2_crimson'

nodes = edges[['victim', 'color']]
nodes.sort_values(by=['victim', 'color'], inplace=True)
victims = nodes['victim'].values
culprits = edges['culprit'].values

for culprit in culprits:
    if not culprit in victims:
        data = {'victim': [culprit], 'color': ['0_green']}
        df = pd.DataFrame(data)
        nodes = pd.concat([nodes, df])

nodes.sort_values(by=['victim', 'color'], inplace=True)
nodes.drop_duplicates('victim', keep='last', inplace=True)
nodes['color'] = nodes['color'].str.split('_').str[1]

descriptions = deaths.groupby(by=['universe', 'victim']).aggregate({'description':'<br>'.join})
descriptions.reset_index(inplace=True)
descriptions['victim'] += " [" + descriptions['universe'] + "]"
descriptions.drop(columns=['universe'], inplace=True)

nodes = pd.merge(nodes, descriptions, left_on=['victim'], right_on=['victim'], how='outer')

In [ ]:
profiles = pd.DataFrame(columns=['universe', 'character', 'profile'])

In [ ]:
def make_network(universe):
    net = Network(height="100%", width="100%", bgcolor="#111111", directed=True)
    net.force_atlas_2based(overlap=1, damping=0.5)

    current_edges = edges[edges['universe'] == universe]
    current_nodes = nodes[nodes['victim'].str.endswith('[' + universe + ']')]

    sources, targets, direction = current_edges['culprit'], current_edges['victim'], current_edges['direction']
    description = current_edges['description']
    edge_data = zip(sources, targets, direction, description)

    for i in range(len(current_nodes)):
        victim = current_nodes.iloc[i]['victim']
        color = current_nodes.iloc[i]['color']
        label = victim.split(" [")[0]
        description = current_nodes.iloc[i]['description']
        title = label + "<br><br>Character Profile :" + "<br><br><br>"

        if description == description:
            value = description.count('<br>') + 1
            if value != 1:
                plural = 's'
            else:
                plural = ''

            title += str(value) + " Death" + plural + " :<br><br>" + description + "<br><br>"
        else:
            title += ''

        if victim != '':
            font = '15px arial sans-serif ' + color
            net.add_node(victim, label=label, title=title, color=color, font=font)

    for src, dst, dir, des in edge_data:
        if src != '':
            if dir == 'direct':
                net.add_edge(src, dst, title=des, color='#e8ac87', width=1, dashes=False)
            elif dir == 'indirect':
                net.add_edge(src, dst, title=des, color='#cccccc', width=1, dashes=True)

    neighbor_map = net.get_adj_list()

    for node in net.nodes:
        value = len(neighbor_map[node['id']])
        if value != 0:
            if value != 1:
                plural = 's'
            else:
                plural = ''
            node['title'] += str(value) + " Victim" + plural + " :<br><br>"
            node['title'] += "<br>".join(sorted(neighbor_map[node['id']]))
            node['title'] = re.sub(' \[.*?\]', '', node['title'])
        node['value'] = value
    
    net.inherit_edge_colors(False)
    net.set_edge_smooth('dynamic')

    net.save_graph('Networks/' + universe + '.html')

    for node in net.nodes:
        universe = node['id'].split(' [')[1].split(']')[0]
        character = node['id'].split(' [')[0]
        profile = node['title']
        row = [universe, character, profile]
        profiles.loc[len(profiles)] = row

In [ ]:
for universe in kills['universe'].unique():
    make_network(universe)

In [ ]:
profiles.to_csv('profiles.tsv', sep='\t', index=False)